# Solar Analytics - Feature Engineering

This notebook creates features for solar power forecasting models.

In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os

load_dotenv()

# Database connection
db_url = f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}/solar_analytics"
engine = create_engine(db_url)

## 1. Create Time-based Features

In [ ]:
def create_time_features(df, timestamp_col='timestamp'):
    """Create time-based features from timestamp"""
    df = df.copy()
    
    # Convert to datetime
    df[timestamp_col] = pd.to_datetime(df[timestamp_col])
    
    # Extract time features
    df['hour'] = df[timestamp_col].dt.hour
    df['day_of_year'] = df[timestamp_col].dt.dayofyear
    df['month'] = df[timestamp_col].dt.month
    df['day_of_week'] = df[timestamp_col].dt.dayofweek
    
    # Solar position features
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['day_sin'] = np.sin(2 * np.pi * df['day_of_year'] / 365)
    df['day_cos'] = np.cos(2 * np.pi * df['day_of_year'] / 365)
    
    return df

# Test with sample data
sample_df = pd.DataFrame({
    'timestamp': pd.date_range('2024-01-01', periods=48, freq='H')
})

sample_with_features = create_time_features(sample_df)
print("Time features created:")
print(sample_with_features.head())

## 2. Create Solar Features

In [ ]:
def create_solar_features(df):
    """Create solar-specific features"""
    df = df.copy()
    
    # Clear sky index (if GHI available)
    if 'ghi' in df.columns and 'solar_ghi' in df.columns:
        df['clear_sky_index'] = df['ghi'] / (df['solar_ghi'] + 0.1)  # Avoid division by zero
    
    # Performance ratio
    if 'ac_power' in df.columns and 'poa_irradiance' in df.columns:
        system_capacity = 4000  # 4kW system
        df['performance_ratio'] = df['ac_power'] / (df['poa_irradiance'] * system_capacity / 1000)
    
    # Lagged features
    for lag in [1, 2, 3, 6, 12, 24]:
        if 'ac_power' in df.columns:
            df[f'power_lag_{lag}h'] = df['ac_power'].shift(lag)
        if 'temperature' in df.columns:
            df[f'temp_lag_{lag}h'] = df['temperature'].shift(lag)
    
    # Rolling averages
    for window in [3, 6, 12, 24]:
        if 'ac_power' in df.columns:
            df[f'power_ma_{window}h'] = df['ac_power'].rolling(window).mean()
    
    return df

print("Solar feature functions created")

## 3. Create Feature Pipeline

In [ ]:
def create_feature_table():
    """Create feature table in mart schema"""
    
    # Load data
    query = """
    SELECT 
        timestamp,
        site_id,
        ac_power,
        dc_power,
        poa_irradiance,
        ambient_temp
    FROM api_ingest.nrel_pvdaq
    WHERE site_id = 'PVWATTS_SIM'
    ORDER BY timestamp
    """
    
    df = pd.read_sql(query, engine)
    
    if len(df) > 0:
        # Add time features
        df = create_time_features(df)
        
        # Add solar features
        df = create_solar_features(df)
        
        # Save to mart schema
        df.to_sql('solar_features', engine, schema='mart', 
                 if_exists='replace', index=False)
        
        print(f"✅ Created feature table with {len(df)} records")
        print(f"Features: {list(df.columns)}")
        
        return df
    else:
        print("No data available for feature engineering")
        return None

# Run feature pipeline
feature_df = create_feature_table()